This is the aircraft gate assignment problem for the Operations optimization course. 

In [12]:
# Gurobi for optimization
from gurobipy import GRB, Model, quicksum
import gurobipy as gp

# Pandas for data manipulation
import pandas as pd

# Numpy for numerical operations
import numpy as np

# Matplotlib for visualization
import matplotlib.pyplot as plt

# Datetime for handling time-related data
import datetime

# Pickle for saving/loading objects (optional, based on need)
import pickle



In [13]:
# Parameters for simulation
num_flights = 50
num_gates = 10

# Generate flight data
np.random.seed(42)
flight_ids = [f"F{i+1}" for i in range(num_flights)]
arrival_times = np.random.randint(0, 1440, num_flights)  # minutes from midnight
departure_times = arrival_times + np.random.randint(60, 180, num_flights)  # ensure at least 1 hour turnaround
passenger_numbers = np.random.randint(50, 300, num_flights)
flight_types = np.random.choice(['domestic', 'international'], num_flights)

flight_data = pd.DataFrame({
    'flight_id': flight_ids,
    'arrival_time': arrival_times,
    'departure_time': departure_times,
    'passenger_number': passenger_numbers,
    'flight_type': flight_types
})

# Generate gate data
gate_ids = [f"G{i+1}" for i in range(num_gates)]
gate_types = np.random.choice(['domestic', 'international'], num_gates)

gate_data = pd.DataFrame({
    'gate_id': gate_ids,
    'gate_type': gate_types
})

# Calculate distances between gates (d_kl) and from gates to entrance/exit (ed_k)
distances = np.random.randint(100, 1000, (num_gates, num_gates))
np.fill_diagonal(distances, 0)
distance_df = pd.DataFrame(distances, index=gate_ids, columns=gate_ids)
ed_k = np.random.randint(100, 500, num_gates)

# Generate transiting passengers data (p_ij)
p_ij = np.random.randint(0, 50, (num_flights, num_flights))
np.fill_diagonal(p_ij, 0)  # No self-transit

# Verify generated data
print("Flight Data:")
print(flight_data.head())

print("\nGate Data:")
print(gate_data.head())

print("\nDistance Matrix (d_kl):")
print(distance_df)

print("\nDistance to Entrance/Exit (ed_k):")
print(pd.Series(ed_k, index=gate_ids))

print("\nTransiting Passengers (p_ij):")
print(pd.DataFrame(p_ij, index=flight_ids, columns=flight_ids).head())


Flight Data:
  flight_id  arrival_time  departure_time  passenger_number    flight_type
0        F1          1126            1274               131  international
1        F2           860             979               160  international
2        F3          1294            1367               102  international
3        F4          1130            1303                73       domestic
4        F5          1095            1163               203       domestic

Gate Data:
  gate_id      gate_type
0      G1  international
1      G2  international
2      G3  international
3      G4       domestic
4      G5  international

Distance Matrix (d_kl):
      G1   G2   G3   G4   G5   G6   G7   G8   G9  G10
G1     0  655  439  897  430  739  605  447  572  330
G2   289    0  484  476  382  732  727  844  358  458
G3   809  555    0  748  417  776  324  918  333  783
G4   763  926  473    0  707  571  332  791  212  929
G5   596  541  663  367    0  906  485  486  212  712
G6   724  180  798  212  1

### Model Explanation

The objective of the FC formulation is to minimize the total walking distance of passengers at the airport, considering both transfer passengers and non-transfer passengers. The model includes the following key components:

- **Parameters**:
  - `num_flights`: Number of flights
  - `num_gates`: Number of gates
  - `arrival_times`: Arrival times of flights
  - `departure_times`: Departure times of flights
  - `passenger_numbers`: Number of passengers per flight
  - `flight_types`: Type of flight (domestic/international)
  - `gate_types`: Type of gate (domestic/international)
  - `distances`: Distance matrix between gates

- **Decision Variables**:
  - `x[i, k]`: Binary variable, 1 if flight \(i\) is assigned to gate \(k\), 0 otherwise
  - `w[i, k, l]`: Continuous variable representing the number of passengers from flight \(i\) through gate \(k\) to gate \(l\)

- **Objective Function**:
  - Minimize the total walking distance of passengers.

- **Constraints**:
  - Each flight must be assigned to exactly one gate.
  - Gates cannot be double-booked.
  - Flow balance constraints for transfer passengers.


In [14]:
# Initialize the Gurobi model
model = gp.Model("FC_Formulation")

# Parameters
num_flights = len(flight_data)
num_gates = len(gate_data)

# Assign parameters from simulated data
d_kl = np.zeros((num_gates + 1, num_gates + 1))
d_kl[:num_gates, :num_gates] = distance_df.values
ed_k = np.append(ed_k, 0)  # Append zero for the apron distance

# Passengers data
e_i = flight_data['passenger_number'].values  # Assuming all passengers come from the entrance
f_i = flight_data['passenger_number'].values  # Assuming all passengers leave after arrival

# Aircraft type (domestic/international)
g_i = flight_data['flight_type'].apply(lambda x: 'D' if x == 'domestic' else 'I').values

# Sets of domestic and international gates
K_D = [k for k, t in zip(range(num_gates), gate_types) if t == 'domestic']
K_I = [k for k, t in zip(range(num_gates), gate_types) if t == 'international']
K_D.append(num_gates)  # Add apron to domestic gates
K_I.append(num_gates)  # Add apron to international gates

# Number of gates
m = len(gate_ids)
NA_star = 5  # Example value, should be defined based on problem

# Transiting passengers (example: random values)
p_ij = p_ij

# Decision Variables
x = model.addVars(num_flights, num_gates + 1, vtype=GRB.BINARY, name="x")
w = model.addVars(num_flights, num_gates, num_gates + 1, vtype=GRB.CONTINUOUS, name="w")

# Verify variables
print("Decision variables defined.")


Decision variables defined.


In [15]:
# Objective function components
objective = quicksum(d_kl[k, l] * w[i, k, l] for i in range(num_flights) for k in range(num_gates) for l in range(num_gates + 1)) + \
            quicksum((e_i[i] + f_i[i]) * ed_k[k] * x[i, k] for i in range(num_flights) for k in range(num_gates + 1))

model.setObjective(objective, GRB.MINIMIZE)

# Verify objective
print("Objective function defined.")

Objective function defined.


In [23]:
# Each aircraft must be assigned to one gate (either a fixed gate or the apron) (Constraint 3)
for i in range(num_flights):
    model.addConstr(quicksum(x[i, k] for k in range(num_gates + 1)) == 1, name=f"aircraft_assignment_{i}")

# Each gate can accommodate at most one aircraft at any time for domestic flights (Constraint 4)
for t in range(num_flights):  # Simplified assumption for time intervals
    for k in K_D:
        model.addConstr(quicksum(x[i, k] for i in range(num_flights) if flight_data['flight_type'][i] == 'domestic') <= 1, name=f"gate_capacity_domestic_{t}_{k}")

# Each gate can accommodate at most one aircraft at any time for international flights (Constraint 5)
for t in range(num_flights):  # Simplified assumption for time intervals
    for k in K_I:
        model.addConstr(quicksum(x[i, k] for i in range(num_flights) if flight_data['flight_type'][i] == 'international') <= 1, name=f"gate_capacity_international_{t}_{k}")

# A minimum number of aircraft must be assigned to the apron (Constraint 6)
model.addConstr(quicksum(x[i, num_gates] for i in range(num_flights)) == NA_star, name="min_apron_assignments")

# Binary constraint for gate assignment (Constraint 7)
for i in range(num_flights):
    for k in range(num_gates + 1):
        model.addConstr(x[i, k] <= 1, name=f"binary_assignment_{i}_{k}")

# Passenger flow balance constraints (Constraints 10 and 11)
for i in range(num_flights):
    for k in range(num_gates + 1):
        model.addConstr(quicksum(w[i, k, l] for l in range(num_gates + 1)) == x[i, k] * sum(p_ij[i, j] for j in range(num_flights)), name=f"flow_balance_out_{i}_{k}")
    for l in range(num_gates + 1):
        model.addConstr(quicksum(w[i, k, l] for k in range(num_gates)) == sum(p_ij[j, i] * x[j, l] for j in range(num_flights)), name=f"flow_balance_in_{i}_{l}")

# Non-negativity constraint for passenger flows (Constraint 12)
for i in range(num_flights):
    for k in range(num_gates + 1):
        for l in range(num_gates + 1):
            model.addConstr(w[i, k, l] >= 0, name=f"non_negativity_{i}_{k}_{l}")

# Verify constraints
print("Constraints defined.")


KeyError: (0, 10, 0)

In [ ]:
# Optimize the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("\nOptimal objective value:", model.ObjVal)
    print("\nAircraft assignments to gates:")
    for i in range(num_flights):
        for k in range(num_gates + 1):
            if x[i, k].X > 0.5:  # Binary variable should be either 0 or 1
                print(f"Flight {flight_ids[i]} assigned to gate {gate_ids[k] if k < num_gates else 'apron'}")

    print("\nPassenger flow between gates:")
    for i in range(num_flights):
        for k in range(num_gates):
            for l in range(num_gates + 1):
                if w[i, k, l].X > 1e-6:  # Only print significant flows
                    print(f"{w[i, k, l].X:.2f} passengers from flight {flight_ids[i]} transfer from gate {gate_ids[k]} to gate {gate_ids[l] if l < num_gates else 'apron'}")
else:
    print("No optimal solution found.")
